In [54]:
import numpy as np
import cv2
import mediapipe as mp
import pandas as pd
import os
import math

In [55]:
mp_face_mes = mp.solutions.face_mesh
mp_drawing = mp.solutions.drawing_utils
drawing_spec = mp_drawing.DrawingSpec(thickness=1, circle_radius=1)
face_mesh = mp_face_mes.FaceMesh(max_num_faces=1, refine_landmarks=True, min_detection_confidence=0.5, min_tracking_confidence=0.5)

In [56]:
images_open = os.listdir('mouth_open')
images_closed = os.listdir('mouth_closed')

print(len(images_open), len(images_closed))

169 206


In [57]:
def update_mesh_points(image):
    try:
        frame = image.copy()
    except AttributeError:
        return None
    H, W, _ = frame.shape
    results_mesh = face_mesh.process(frame)
    if results_mesh.multi_face_landmarks:
        mesh_points = np.array([np.multiply([p.x, p.y, p.z], [W, H, max(W, H)]).astype(int) for p in results_mesh.multi_face_landmarks[0].landmark])
        return mesh_points
    return None

In [58]:
MOUTH = [78, 191, 80, 81, 82, 13, 312, 311, 310, 415, 308, 324, 318, 402, 317, 14, 87, 178, 88, 95]

def calculate_mouth_area(mesh_points):
    ex1, ex2 = mesh_points[78], mesh_points[308]
    area = 0
    for i in range(len(MOUTH) - 1):
        area += (mesh_points[MOUTH[i], 0] * mesh_points[MOUTH[i + 1], 1]) - (mesh_points[MOUTH[i + 1], 0] * mesh_points[MOUTH[i], 1])
    area = abs(area / (2*(math.sqrt((ex1[0]-ex2[0])**2 + (ex1[1]-ex2[1])**2)**2)))
    return area

def calculate_width_over_height(mesh_points):
    ex1 = mesh_points[78]
    ex2 = mesh_points[308]
    ey1 = mesh_points[13]
    ey2 = mesh_points[14]
    try:
        reason = math.sqrt((ex1[0]-ex2[0])**2+(ex1[1]-ex2[1])**2+(ex1[2]-ex2[2])**2)/math.sqrt((ey1[0]-ey2[0])**2+(ey1[1]-ey2[1])**2+(ey1[2]-ey2[2])**2)
        if reason > 100: reason = 100
    except ZeroDivisionError:
        reason = 100
    return reason

def get_features(frame):
    mesh_points = update_mesh_points(frame)
    if mesh_points is None:
        return None
    area = calculate_mouth_area(mesh_points)
    reason = calculate_width_over_height(mesh_points)
    return (area, reason)

In [59]:
def resize_if_too_small(photo):
    if photo.shape[0] < 300 and photo.shape[1] < 300:
        photo = cv2.resize(photo, (300, 300), interpolation = cv2.INTER_CUBIC)
    return photo

In [60]:
results_open = []
results_closed = []

print('Processing open images...')
for image in images_open:
    frame = cv2.imread('mouth_open/'+image)
    if frame is None:
        continue
    frame = resize_if_too_small(frame)
    result = get_features(frame)
    if result:
        results_open.append((result[0], result[1]))

print('Processing closed images...')
for image in images_closed:
    frame = cv2.imread('mouth_closed/'+image)
    if frame is None:
        continue
    frame = resize_if_too_small(frame)
    result = get_features(frame)
    if result:
        results_closed.append((result[0], result[1]))

Processing open images...
Processing closed images...


In [61]:
print(len(results_open), len(results_closed))

104 129


In [62]:
results_open = np.array(results_open)
results_closed = np.array(results_closed)

In [63]:
df = pd.DataFrame(columns=['normalized_mouth_area', 'width_over_height', 'label'])

for value in results_open:
    df.loc[len(df)] = {'normalized_mouth_area': value[0], 'width_over_height': value[1], 'label': 'open'}
for value in results_closed:
    df.loc[len(df)] = {'normalized_mouth_area': value[0], 'width_over_height': value[1], 'label': 'closed'}

df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 233 entries, 0 to 232
Data columns (total 3 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   normalized_mouth_area  233 non-null    float64
 1   width_over_height      233 non-null    float64
 2   label                  233 non-null    object 
dtypes: float64(2), object(1)
memory usage: 7.3+ KB


In [64]:
df

,normalized_mouth_area,width_over_height,label
0,0.325770,1.928138,open
1,0.495050,100.000000,open
2,0.183761,15.427249,open
3,0.381148,100.000000,open
4,0.383047,100.000000,open
...,...,...,...
228,0.240685,48.301139,closed
229,0.161975,100.000000,closed
230,0.203498,25.573424,closed
231,0.281815,55.154329,closed


In [65]:
df.to_json('data.json', index=False)